# Session 1: part 2
<a href="https://colab.research.google.com/github/ntu-dl-bootcamp/deep-learning-2024/blob/main/session1/session1_p2_student.ipynb" target="_blank"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>


We will apply the knowledge from previous session to analyze a dataset of houses. This dataset is obtained from the Coursera machine learning foundations course (https://www.coursera.org/learn/ml-foundations/supplement/RP8te/predicting-house-prices-assignment), a highly recommended course. It can be downloaded at https://d396qusza40orc.cloudfront.net/phoenixassets/home_data.csv.

__Exercises__: Just follow along this notebook, exercises are indicated with exceptions (`raise` keyword). You are also invited to play with the code, discuss with your colleagues and ask questions to the instructors.

First we load the data and import the libraries we will use.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Download the file
!rm -f home_data.csv
!wget https://raw.githubusercontent.com/ntu-dl-bootcamp/deep-learning-2024/main/session1/home_data.csv
# Load home_data.csv into a dataframe
homes = pd.read_csv('home_data.csv')
homes.head()

We can see that we have 21 features and 21613 samples.

In [ ]:
homes.shape

# Data analysis

Now that we have the data loaded, we can start exploring it. 

In [ ]:
# Plot a histogram of price
raise NotImplementedError("TODO: plot a histogram of price")

plt.___(homes['price'])
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Histogram of Price')

In [ ]:
import matplotlib.pyplot as plt

# Plot square feet vs price of house
raise NotImplementedError("TODO: plot square feet vs price of house")
plt.___(homes['sqft_living'], homes['price'])
plt.xlabel('Living Area (sqft)')
plt.ylabel('Price ($)')
plt.title('Price vs Living Area')

In [ ]:
# Also plot bedrooms and bathrooms vs price
# TODO Find out how to add a legend
raise NotImplementedError("TODO: add a legend in the below plot")
plt.scatter(homes['bedrooms'], homes['price'], ___)
plt.scatter(homes['bathrooms'], homes['price'], ___)
plt.xlabel('Feature')
plt.ylabel('Price ($)')
plt.title('Price vs Feature')
___

In [ ]:
# Exercise. generate summaries of the data for all columns except id 
# Is there a pandas API that seems helpful? 
# Check the docs at https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html
raise NotImplementedError("TODO: generate summaries of the data for all columns except id")

In [ ]:
# Exercise. Find the most expensive house and the biggest house
raise NotImplementedError("TODO: find the most expensive house and the biggest house")

In [ ]:
# Exercise. Find the average price, the median price, and the average price of a 3 bedroom house
raise NotImplementedError("TODO: Find the average price, the median price, and the average price of a 3 bedroom house")

# Linear regression

We will now use a linear regression model to predict the housing prices. As in the typical machine learning workflow, we perform the following steps:
1. Split the data into training and test sets
2. Train the model on the training set
3. Evaluate the model on the test set

We will use the scikit-learn library to perform the linear regression. Scikit-learn is a very popular library for machine learning in Python. It provides a wide range of machine learning algorithms and tools for data analysis and visualization. It is also very well documented, so it is a good idea to get familiar with it.

For linear regression, we can use the `LinearRegression` class from scikit-learn. We will use the `fit` method to train the model and the `predict` method to make predictions.

In [ ]:
# set seed for reproducibility
np.random.seed(0)
train_homes, test_homes = train_test_split(homes, test_size=0.2)

In [ ]:
# we can check the size of the train and test sets
train_homes.shape
test_homes.shape

In [ ]:
# build linear regression model from sqft_living to price

# fit the model
sqft_model = LinearRegression()
sqft_model.fit(train_homes[['sqft_living']], train_homes['price'])

# make predictions on the test data
pred_homes = sqft_model.predict(test_homes[['sqft_living']])
pred_homes.shape

In [ ]:
# Now we can analyze the model performance.

# The coefficients
print("Coefficients:", sqft_model.coef_)
# The mean squared error
print("Mean squared error: %.2f" %  np.sqrt(mean_squared_error(test_homes['price'], pred_homes)))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(test_homes['price'], pred_homes))

In [ ]:
# plot the predictions and the original data
plt.scatter(test_homes['sqft_living'], test_homes['price'])
plt.plot(test_homes['sqft_living'], pred_homes, color='red')

We achieve a relatively satisfactory result. Let's see how we can improve it. Perhaps we can use more features? Try to add a few more features and compare the results. Can you improve the performance?

In [ ]:
raise NotImplementedError("TODO: improve the model by adding more features")

__Discuss__: 
1. What is the difference between the training and test sets? Why do we need both?
2. Why is the `multi_model` performing better than the original `sqft_model`?


# Linear Regression with Pytorch

Now we will use the same dataset to perform linear regression using PyTorch. We will use the same features as before, but we will use a different model. Instead of using a linear model, we will use a neural network with no hidden layer. 

Do not try to understand all the details of the code below. The important thing is to understand the general structure of the code and the main steps of the training process:
1. Load the data
2. Define the model, the loss function and the optimizer. These concepts will be introduced in more details in the next session. For now, just think of the model as a function that takes the input and returns the output. The loss function is a function that measures how good the model is. The optimizer is an algorithm that tries to minimize the loss function.
5. Train the model for a number of epochs. Train consists of using the loss function to measure the error of the model and the optimizer to update the parameters of the model to reduce the error.
6. Evaluate the model. Reuse the loss function to measure the error of the model on the test set.



In [ ]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import random
import os

def make_training_deterministic(seed: int = 0):
    '''Set random seed for reproducibility'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
# Make the model training deterministic
make_training_deterministic(0)

# Load the data
data = homes[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']]
scaler = StandardScaler()
X = torch.tensor(scaler.fit_transform(data), dtype=torch.float32)
y = torch.tensor(homes['price'].values.reshape(-1, 1), dtype=torch.float32)

# train-test split for model evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

In [ ]:
# define the model 
model = nn.Linear(X.shape[1], y.shape[1])

# define the loss function
loss_fn = nn.MSELoss()

# define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01) 

In [ ]:
# The training loop
n_epochs = 500 # number of epochs to run
train_loss = []
for epoch in range(n_epochs):
    model.train() # set model to training mode
    # forward pass
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    # update weights
    optimizer.step()
    # save loss for plotting
    train_loss.append(loss.item())

In [ ]:
# The test loop
model.eval() # set model to evaluation mode
y_pred = model(X_test)
loss = loss_fn(y_pred, y_test)
print("Root mean squared error: %.2f" % np.sqrt(loss.item()))
print("r2_score: %.2f" % r2_score(y_test.detach().numpy(), y_pred.detach().numpy()))

In [ ]:
# plot the train loss
plt.plot(train_loss)

Note that we are able to achieve very similar results using PyTorch. It turns out that a one layer neural network is equivalent to a linear regression model. You can read more about this on https://deeplearning.neuromatch.io/tutorials/W1D2_LinearDeepLearning/student/W1D2_Tutorial3.html#bonus (see Bonus section).